## Author: Isaac Coffie
## Course: NLP
## Date: April 26, 2020
## Assignment: POS Tagging Using LSTM

In [ ]:
##Import libraries
import numpy as np
import pandas as pd

##preprocessing utils
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


## Model fitting utils
from keras.models import Sequential
from keras.layers import InputLayer, Embedding, LSTM, Bidirectional, TimeDistributed, Dense, Activation
from keras.optimizers import Adam

In [ ]:
from pickle import load
from pickle import dump


## Some helper functions

In [ ]:
## define helper functions
## converts the softmax output to a POS tag
def convert_logits_to_class_labels(model_pred_tags, index_to_tag_dict, limit):
  counter = 0
  predicted_pos_tag = list()
  for y_hat in model_pred_tags:
    if counter == limit:
      break
    predicted_pos_tag.append(index_to_tag_dict[y_hat])
    counter +=1
  return np.asarray(predicted_pos_tag)

## converts a sequence of words into a sequence of integers using the sentence to index dictionary
def convert_sentence_to_integer_sequence(sentence_to_index_dict, sentence):
    to_return = []
    for word in sentence:
        try:
            to_return.append(sentence_to_index_dict[word])
        except KeyError:
            to_return.append(sentence_to_index_dict['OOV'])
    return np.asarray([to_return])

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)
 
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

## Load Dataset

In [ ]:
##Mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
## change directory to point to the Google drive folder that contains the dataset
%cd "/content/drive/My Drive/NLP_Class"
!pwd

/content/drive/My Drive/NLP_Class
/content/drive/My Drive/NLP_Class


In [ ]:

training_data = "/sentence_pos_training.csv"
input_path = "/content/drive/My Drive/NLP_Class"


training_df = pd.read_csv(input_path + training_data)


In [ ]:
## extract useful columns
training_df = training_df.iloc[:, 1:]
training_df.head()


local_train = training_df.iloc[:,:]

##maintain only setnece with more than two tokens or words
local_train = local_train[local_train.groupby('sentence_id').token_id.transform(len) > 2]
local_train.tail(10)


,sentence_id,token_id,pos_tag
4107505,748062,in,IN
4107506,748062,the,DT
4107507,748062,Morbihan,NN
4107508,748062,department,NN
4107509,748062,of,IN
4107510,748062,Brittany,NNP
4107511,748062,in,IN
4107512,748062,north,RB
4107513,748062,western,JJ
4107514,748062,France,NNP


## Creating Sequences of Sentences

In [ ]:
unique_sentences_train = np.unique(local_train.sentence_id.values)

sentences_sequence = list()
sentences_tagged_sequence = list()

for sentence in unique_sentences_train:

    this_sentence_record_new = local_train.loc[local_train["sentence_id"] == sentence]
    sent_flattened = this_sentence_record_new["token_id"].values.flatten()
    pos_tag = this_sentence_record_new["pos_tag"].values.flatten()

    sentences_sequence.append(sent_flattened)
    sentences_tagged_sequence.append(pos_tag)
    print(sentence)

Streaming output truncated to the last 5000 lines.
740255
740256
740257
740258
740259
740260
740261
740263
740265
740267
740269
740270
740271
740272
740273
740275
740276
740277
740278
740279
740280
740282
740283
740284
740288
740289
740290
740292
740293
740295
740296
740298
740299
740301
740302
740303
740304
740305
740306
740310
740311
740313
740314
740315
740316
740317
740319
740323
740327
740329
740331
740332
740333
740334
740335
740336
740338
740342
740344
740346
740348
740349
740351
740352
740353
740359
740361
740362
740363
740364
740365
740367
740368
740370
740373
740374
740379
740382
740384
740387
740388
740389
740390
740391
740393
740396
740397
740400
740404
740405
740407
740409
740410
740411
740414
740415
740416
740417
740420
740422
740424
740427
740428
740429
740431
740432
740433
740434
740435
740436
740438
740439
740440
740441
740442
740443
740444
740445
740446
740448
740450
740452
740453
740454
740455
740456
740458
740460
740462
740464
740465
740467
740470
740471
740472
7404

### Flatten the sentence array above

In [ ]:

flattend_sentence = np.concatenate(sentences_sequence).ravel().tolist()
flattend_tags = np.concatenate(sentences_tagged_sequence).ravel().tolist()

## Account for instance where our training corpus doesn't have words in the test data
flattend_sentence.append("OOV")  ## oov for out of vocabulary
##flattend_tags.append("OOV_TAG")

### Convert the words into a integer values.. necessary for embedding

In [ ]:
tags_to_index_encoded = {tag: index for index, tag in enumerate(np.unique(flattend_tags))}

index_to_tag_encoded = {index: tag for index, tag in enumerate(np.unique(flattend_tags))}

sentence_to_index_encoded = {token: index for index, token in enumerate(np.unique(flattend_sentence))}

del tags_to_index_encoded["nan"] ## delete this unknown tag

print("size of vocabularly", len(sentence_to_index_encoded))


size of vocabularly 239087


In [ ]:
save_clean_data(sentence_to_index_encoded, 'sentence_to_index_encoded.pkl')
save_clean_data(tags_to_index_encoded, 'tags_to_index_encoded.pkl')
save_clean_data(index_to_tag_encoded, 'index_to_tag_encoded.pkl')



Saved: sentence_to_index_encoded.pkl
Saved: tags_to_index_encoded.pkl
Saved: index_to_tag_encoded.pkl


## Vectorize the sequence of words in a sentence using their corresponding index number in the above dictionary

In [ ]:
## Start with the sentences
sentences_vectorized = list()

for entry in sentences_sequence:
    this_sent = list()
    for token in entry:

      try:
        this_sent.append(sentence_to_index_encoded[token])
      except KeyError:
        this_sent.append(sentence_to_index_encoded['OOV'])
 
    sentences_vectorized.append(this_sent)

sentences_vectorized[15]

[167425,
 179940,
 178860,
 167686,
 126373,
 2725,
 50567,
 209995,
 227640,
 197079,
 158849,
 232074,
 186676,
 195137,
 226580]

In [ ]:
## vectorize the sequence of tags as well
tags_vectorized = list()
import random

arr = ["NN", "VBG", "JJ", "SYM"]


for tags in sentences_tagged_sequence:

  this_tag = list()
  for tag in tags:
    try:
      this_tag.append(tags_to_index_encoded[tag])

    except KeyError:
        random_tag = random.choice(arr)
        this_tag.append(tags_to_index_encoded[random_tag])

  tags_vectorized.append(this_tag)

### Pad sequences to max length

In [ ]:
MAX_SEQUENCE = 25
sentences_vectorized_padded = pad_sequences(sentences_vectorized, maxlen=MAX_SEQUENCE, padding='post')
tags_vectorized_padded = pad_sequences(tags_vectorized, maxlen=MAX_SEQUENCE, padding='post')

In [ ]:
## Save the cleaned file

save_clean_data(sentences_vectorized_padded, 'sentences_vectorized_padded.pkl')
save_clean_data(tags_vectorized_padded, 'tags_vectorized_padded.pkl')

NameError: ignored

## Building the model

In [ ]:
## add the structure
model = Sequential()

## add the input layer
model.add(InputLayer(input_shape=(MAX_SEQUENCE, ))) 
## add the embedding layer of size 200 vectors
model.add(Embedding(input_dim= len(sentence_to_index_encoded), output_dim=200, input_length = MAX_SEQUENCE))
## add the bidirectional LSTM model with 128 neurons or units
model.add(Bidirectional(LSTM(128, return_sequences=True)))
## add the fully connected dense layer
model.add(TimeDistributed(Dense(len(tags_to_index_encoded))))
## add the softmax activation function
model.add(Activation('softmax'))
## compile the model using these lose values 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 200)           47817400  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 256)           336896    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 25, 88)            22616     
_________________________________________________________________
activation_1 (Activation)    (None, 25, 88)            0         
Total params: 48,176,912
Trainable params: 48,176,912
Non-trainable params: 0
_________________________________________________________________


### Converting pos tags to categorical for the dense layer

In [ ]:
## convert the POS tags to one hot categorical encoding
encoded = to_categorical(tags_vectorized_padded)
model.fit(sentences_vectorized_padded, encoded, batch_size=200, epochs=40, validation_split=0.2)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 308758 samples, validate on 77190 samples
Epoch 1/40
308758/308758 [==============================] - 234s 757us/step - loss: 0.2532 - accuracy: 0.9338 - val_loss: 0.0635 - val_accuracy: 0.9790
Epoch 2/40
308758/308758 [==============================] - 231s 747us/step - loss: 0.0393 - accuracy: 0.9864 - val_loss: 0.0584 - val_accuracy: 0.9807
Epoch 3/40
308758/308758 [==============================] - 232s 750us/step - loss: 0.0251 - accuracy: 0.9909 - val_loss: 0.0605 - val_accuracy: 0.9806
Epoch 4/40
308758/308758 [==============================] - 232s 751us/step - loss: 0.0180 - accuracy: 0.9936 - val_loss: 0.0663 - val_accuracy: 0.9807
Epoch 5/40
308758/308758 [==============================] - 232s 751us/step - loss: 0.0130 - accuracy: 0.9954 - val_loss: 0.0729 - val_accuracy: 0.9798
Epoch 6/40
308758/308758 [==============================] - 231s 748us/step - loss: 0.0093 - accuracy: 0.9968 - val_loss: 0.0815 - val_accuracy: 0.9792
Epoch 7/40
308758/308758 [===========

## Evaluate and save model

In [ ]:
model.evaluate(sentences_vectorized_padded, encoded)

# save model and architecture to single file
model.save("coffie_model_latest.h5")
print("Successfully saved model to file")


385948/385948 [==============================] - 139s 361us/step
Successfully saved model to file


In [ ]:
model.evaluate(sentences_vectorized_padded, encoded)


385948/385948 [==============================] - 138s 358us/step


[0.03857001174768763, 0.9952389597892761]

## Test the model

In [ ]:
from keras.models import load_model

MAX_SEQUENCE = 25

## load model
model = load_model('coffie_model_latest.h5')

# summarize model ensure its still the one that we saved
print(model.summary())

## get test file names... please change the file names to the correct test file
test_file_name = "test_sentences_hw02.csv" ## change here
test_file_output_name = "coffie_pos_2.csv"

## read test file.. i assumed the format of the test file is tab a delimited .txt file. Please change it to the correct format using the sep paramter
testing_df = pd.read_csv(test_file_name)
unique_sentences_test = np.unique(testing_df.sentence_id.values)
test_sentences_sequence = list()
out_pos_test = pd.DataFrame()

sentence_to_index_encoded = load_clean_sentences('sentence_to_index_encoded.pkl')
index_to_tag_encoded = load_clean_sentences('index_to_tag_encoded.pkl')


for sentence in unique_sentences_test:
    print(sentence)

    this_sentence_record_new = testing_df.loc[testing_df["sentence_id"] == sentence]
    sent_flattened = this_sentence_record_new["before"].values.flatten()
    sent_tokens = this_sentence_record_new["token_id"].values.flatten()


    ## convert_sentence_to_integer_sequence
    sequence_data = convert_sentence_to_integer_sequence(sentence_to_index_encoded, sent_flattened)
    
    ## pad sequence to 25
    sequence_data_padded = pad_sequences(sequence_data, maxlen=MAX_SEQUENCE, padding='post')

    print(sent_flattened)
    sent_len = len(sent_flattened)
    if(sent_len > 25):
      sent_len = MAX_SEQUENCE
      sent_tokens = sent_tokens[0:25]


    
    ## Make prediction
    predicted_labels_prob = model.predict_classes(sequence_data_padded)
    predicted_labels = convert_logits_to_class_labels(predicted_labels_prob[0], index_to_tag_encoded, sent_len)

    ## format the predicted labels correctly
    inner_df = pd.DataFrame({"sentence_id": np.repeat(sentence, sent_len), "token_id": sent_tokens, "pos": predicted_labels})
    out_pos_test = out_pos_test.append(inner_df, ignore_index = True)

## save to csv file. i assumed the required .txt format is also tab delimited
out_pos_test["sentence_id"] = out_pos_test["sentence_id"].astype(int)
out_pos_test["token_id"] = out_pos_test["token_id"].astype(int)
out_pos_test.to_csv(test_file_output_name, index=False)
## this will be saved to the current Google Drive folder... kindly check that


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Streaming output truncated to the last 5000 lines.
['Brigaglia' 'Storia' 'della' 'Sardegna' '(' '1995' ')' 'p' '.43' '.']
740685
['Nigeria' 'has' 'joined' 'the' 'space' 'race' 'Others' 'should' 'join'
 'too' '"' '.']
740695
['Code' 'of' 'Honor' 'consists' 'of' 'an' 'oath' 'and' 'five' 'tenets' '.']
740696
['Barksdale' 'and' 'Stringer' 'Bell' 'meet' 'with' 'Wee' 'Bey' 'Brice'
 'and' 'tell' 'him' 'to' 'contact' 'Omar' 'Little' 'to' 'negotiate' 'a'
 'truce' '.']
740697
['the' '1995' 'Nunavut' 'capital' 'plebiscite' ',' 'Iqaluit' 'defeated'
 'Rankin' 'Inlet' 'to' 'become' 'territorial' 'capital' 'of' 'Nunavut' '.']
740699
['the' 'same' 'time' ',' 'the' 'League' 'was' 'incorporated' 'and'
 'engaged' 'its' 'first' 'Assistant' 'Secretary' 'to' 'increase' 'its'
 'services' 'to' 'members' '.']
740703
['$153,000' 'in' 'college' 'scholarships' 'are' 'awarded' 'at' 'each'
 'national' 'tournament' ',' 'making' 'it' 'possible' 'for' 'students'
 'to' 'pursue' 'post' 'secondary' 'education' '.']
74070

In [ ]:
## joining the 2 dataframes... Because of computing and internet issues I had to divide the test dataset into 2
df_2 = pd.read_csv("coffie_pos_2.csv")
df_1 = pd.read_csv("coffie_pos_1.csv")

df_1 = df_1.iloc[0:-13, :]
final_output = df_1.append(df_2, ignore_index = True)
final_output.to_csv("coffie_pos.csv", index=False)

,sentence_id,token_id,pos
0,1,0,NN
1,1,1,NN
2,1,2,NNP
3,1,3,NN
4,1,4,IN
